## Interleaving computations with a rudimentary event loop

In this example, we define a generator-based `factorial()` function. It does the computation iteratively, without any recursion. Each loop does one multiplication computation, before yielding.

We also define a `step()` procedure (which is simply calls `__next__()` on its input), as well as a rudimentary event loop `run()`. This event loop accepts a list of arguments, which are tuples of (generator, callback to call with the final result of the generator). It also must be configured with a mode, either interleaved or non-interleaved.

The event loop runs through all of the generators. When a generator is completed, it gathers the return value, and passes it to its registered callback. When all generators have completed, `run()` also completes.

When run in non-interleaved mode, the first generator is run uninterrupted until completion, then its callback is called. Then the same is done for the second generator, and so on.

When run in interleaved mode, each generator gets run for exactly one step, before being pushed to the back of the queue and then running a step on the next generator. When a generator is completed, its callback is called immediately, before the next generator is stepped.

In [6]:
from collections import OrderedDict

def factorial(N):
    if not isinstance(N, int):
        raise TypeError("factorial requires an int")
    if N < 0:
        raise ValueError("factorial requires a non-negative int")
    factorial = _factorial(N)
    factorial.__name__ = f"factorial({N})"
    return factorial 

def _factorial(N):
    if 0 <= N <= 1:
        return 1
    if N == 2:
        return 2
    product = N * (N - 1)
    n = N - 2
    while n > 1:
        print("partial product for", f"{N}!", "is", product)
        yield product
        product *= n
        n -= 1
    return product

def step(generator):
    next(generator)

def run(*args, interleave):
    d = OrderedDict(args)
    rounds = 0
    mode = "interleaved" if interleave else "non-interleaved"
    print("Starting", mode, "computations.")
    while d:
        rounds += 1
        generator, on_complete = d.popitem(last=False)
        try:
            step(generator)
        except StopIteration as exc:
            print("Completed", generator.__name__, "after", rounds, "rounds")
            on_complete(generator, exc.value)
        else:
            d[generator] = on_complete
            d.move_to_end(generator, last=interleave)

def print_result(factorial_generator, result):
    print(factorial_generator.__name__, '==', result)
            
run((factorial(12), print_result), (factorial(8), print_result), (factorial(4), print_result), interleave=False)          
print()
run((factorial(12), print_result), (factorial(8), print_result), (factorial(4), print_result), interleave=True)

Starting non-interleaved computations.
partial product for 12! is 132
partial product for 12! is 1320
partial product for 12! is 11880
partial product for 12! is 95040
partial product for 12! is 665280
partial product for 12! is 3991680
partial product for 12! is 19958400
partial product for 12! is 79833600
partial product for 12! is 239500800
Completed factorial(12) after 10 rounds
factorial(12) == 479001600
partial product for 8! is 56
partial product for 8! is 336
partial product for 8! is 1680
partial product for 8! is 6720
partial product for 8! is 20160
Completed factorial(8) after 16 rounds
factorial(8) == 40320
partial product for 4! is 12
Completed factorial(4) after 18 rounds
factorial(4) == 24

Starting interleaved computations.
partial product for 12! is 132
partial product for 8! is 56
partial product for 4! is 12
partial product for 12! is 1320
partial product for 8! is 336
Completed factorial(4) after 6 rounds
factorial(4) == 24
partial product for 12! is 11880
partial p

Both interleaved and non-interleaved have the same total runtimes. However, the program may appear to be more responsive in one mode over the other, depending on the scheduled generators. In non-interleaved mode, the number of rounds to complete a particular generator is T + R, where R is the number of rounds to complete that generator in isolation, and T is the total number of rounds to complete all of the generators that were scheduled before it. Whereas, in interleaved mode, the number of rounds is, at most, (N * R) + R, where N is the number of other generators in the event loop (not including itself). Let's consider how these behave under various conditions:
- One task: No difference.
- One short task followed by one long task: No matter what, the long task will complete at the same time. But the short task will take slightly longer in interleaved mode.
- One long task followed by one short task: The long task will finish slightly faster in non-interleaved mode, but the short task will finish much faster in interleaved mode.
- Many tasks of equal length: In non-interleaved mode, one generator will complete every R rounds. In interleaved mode, a long time will pass without completing any generators, but towards the end many generators will complete in rapid succession.

Interleaved mode isn't guaranteed to be better, but it is probably expected to perform better on average. In this mode, if there are only a few generators, then short tasks will always complete fast. Whereas the more generators there are in the run loop, the slower every generator will run, but each generator is penalized proportionally to its size. Non-interleaved mode can sometimes perform better, but since it penalizes based on position in the queue and based on the sizes of previous generators, it can cause massive and unfair slowdowns.

In the example below, switching from non-interleaved to interleaved mode has the following effects:
- `factorial(12)` goes from completing after 10 rounds, to completing after 18
- `factorial(8)` goes from completing after 16 rounds, to completing after 14
- `factorial(4)` goes from completing after 18 rounds, to completing after 6

# License

License: [Apache License, Version 2.0][Apache License]  
[Jordan Moldow][], 2017

>     Copyright 2017 Jordan Moldow
>
>     Licensed under the Apache License, Version 2.0 (the "License");
>     you may not use this file except in compliance with the License.
>     You may obtain a copy of the License at
>
>         http://www.apache.org/licenses/LICENSE-2.0
>
>     Unless required by applicable law or agreed to in writing, software
>     distributed under the License is distributed on an "AS IS" BASIS,
>     WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
>     See the License for the specific language governing permissions and
>     limitations under the License.

[Jordan Moldow]: <https://github.com/jmoldow> "Jordan Moldow"
[Apache License]: <http://www.apache.org/licenses/LICENSE-2.0> "Apache License, Version 2.0"